# Assignment 2: Deep Q learning

The problem with Q-learning is that the size of the Q-table becomes infeasible for large boards. Let's take it one step further and train a Deep Q model, which can handle more.

**Task**: Import necessary packages. Could take a little more time than with the previous assignment. Watch the asterisk.

In [ ]:
import nbimporter
nbimporter.options['only_defs'] = False
from pacman_game import Action, initialize_gamestate_from_file, get_next_game_state_from_action, ActionEvent
from collections import deque
import numpy as np
from copy import deepcopy
import tensorflow as tf
config = tf.ConfigProto()
# config.gpu_options.allow_growth = True
config.gpu_options.per_process_gpu_memory_fraction = 0.02
from keras.backend.tensorflow_backend import set_session
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import sgd

set_session(tf.Session(config=config))

## Rewards

**Task:** As with Q-learning, specify the reward values

In [ ]:
def calculate_reward_for_move(action_event):
    if action_event == ActionEvent.DOT:
        # TODO
    elif action_event == ActionEvent.CAPTURED_BY_GHOST:
        # TODO
    elif action_event == ActionEvent.NONE:
        return -0.1
    elif action_event == ActionEvent.WALL:
        # TODO
    elif action_event == ActionEvent.WON:
        # TODO
    elif action_event == ActionEvent.LOST:
        # TODO
    return 0

## Model
A possible neural net model is for the most part implemented below. The input is the board, and thus the input_size must correspond with the size of the board (number of board elements). For level 0, this is 60. If you change the board to train on, the input size must also change. The model will use Stochastic Gradient Descent for optimizing loss and Mean-Squared-Error as error function. The convert_state_to_input method converts board elements (pacman, ghost, dot, etc..) to one-hot vectors, and represents the board as one long list of one-hot vectors concatenated together.

**Tasks:** 
1. Assign num_actions. Get the number of actions by getting len() of Action.get_all_actions(). This number corresponds to number of ouput neurons in the network. 


2. Complete the network by adding an output layer to the model after the last hidden layer. Use Dense() which takes number of neurons as argument.


In [ ]:
def convert_state_to_input(state):
    string_rep = state.__str__()
    r = np.array([])

    for char in string_rep:
        if char == 'o':
            r = np.concatenate([r, [0, 0, 0, 0, 1]])
        if char == ' ':
            r = np.concatenate([r, [0, 0, 0, 1, 0]])
        if char == 'P':
            r = np.concatenate([r, [0, 0, 1, 0, 0]])
        if char == 'G':
            r = np.concatenate([r, [0, 1, 0, 0, 0]])
        if char == '.':
            r = np.concatenate([r, [1, 0, 0, 0, 0]])

    return r.reshape(1, r.size)

In [ ]:
## DEFINE MODEL ##
level = 'level-0'
initial_game_state = initialize_gamestate_from_file(level)
input_size = convert_state_to_input(initial_game_state).size

# TODO: Get number of actions
num_actions = 

model = Sequential()  # Input layer is embedded
model.add(Dense(256, input_shape=(input_size,), activation='relu'))  # Hidden layer
model.add(Dense(128, activation='relu'))  # Hidden layer
# TODO: Add output layer.
model.compile(sgd(lr=.01), "mse")


## Actions
This part is the same as with Q-learning.

**Task:** Pick exploration probability.

In [ ]:
def pick_optimal_action(state):
    q = model.predict(convert_state_to_input(state))
    return Action.get_all_actions()[np.argmax(q[0])]

In [ ]:
def pick_action(game_state):
    exploration_prob = 0.20
    if exploration_prob > np.random.rand():
        # Explore
        return np.random.choice(Action.get_all_actions())
    else:
        # Exploit
        return pick_optimal_action(game_state)

## Experience Replay

The experience replay implementation is just procedural and is fully implemented below. The memory is implemented as a deque with a max size. Thus, when the memory is full, new experiences will push out old experiences as in a queue.

The memory is filled with Experience objects.

In [ ]:
class Memory:
    def __init__(self, max_size):
        self.memory = deque(maxlen=max_size)

    def add(self, experience):
        self.memory.append(experience)

    def get(self, index):
        return self.memory[index]

    def get_mini_batch(self, batch_size):
        memory_size = self.get_size()
        indices = np.random.choice(np.arange(memory_size), min(batch_size, memory_size), replace=False)
        return [self.memory[i] for i in indices]

    def get_size(self):
        return len(self.memory)


In [ ]:

class Experience:

    def __init__(self, current_state, action, reward, next_state, done: bool):
        """
        Args:
            done (bool):
            current_state (GameState):
            action (Action):
            reward (int):
            next_state (GameState):
        """
        self.current_state = current_state
        self.action = action
        self.reward = reward
        self.next_state = next_state
        self.done = done


## Training
Most of the intricate details are already implemented.

**Tasks:** 
1. Assign a new Experience object to the experience variable. Use convert_state_to_input to convert the game states before storing them in the Experience.


2. Add the experience to the memory.


3. Get a mini batch to train on and assign to batch


4. Run model.train_on_batch() and pass x_train and np.asarray(y_train) as arguments. Assign this to batch_loss.



In [ ]:
def train(level, num_training_episodes, batch_size, gamma=0.9):
    initial_game_state = initialize_gamestate_from_file(level)
    tot_loss = {}
    memory = Memory(max_size=5000)

    for i in range(1, num_training_episodes):

        loss = 0.
        num_episode_steps = 0

        done = False
        current_game_state = deepcopy(initial_game_state)

        while not done:
            if num_episode_steps > 500:
                break

            action = pick_action(current_game_state)
            next_game_state, action_event = get_next_game_state_from_action(current_game_state, action.name)

            if action_event == ActionEvent.WON or action_event == ActionEvent.LOST:
                done = True
                if action_event == ActionEvent.WON:
                    print("Won!!")
                else:
                    print('lost')

            reward = calculate_reward_for_move(action_event)
            
            # TODO: Store new experience
            experience = 
            # TODO: Add the experience to the memory.
    
            # TODO: Get a mini batch to train on from memory.
            batch = 

            # Dimensions of our observed states, ie, the input to our model.
            input_dim = batch[0].current_state.shape[1]
            x_train = np.zeros((min(memory.get_size(), batch_size), input_dim))
            y_train = np.zeros((x_train.shape[0], len(Action.get_all_actions())))  # Target Q-value

            sample: Experience
            for j, sample in enumerate(batch):
                y_target = model.predict(sample.current_state)[0]

                x_train[j:j + 1] = sample.current_state
                if sample.done:
                    y_target[sample.action.value] = sample.reward
                else:
                    y_target[sample.action.value] = sample.reward + gamma * np.max(model.predict(sample.next_state))
                y_train[j] = y_target

            batch_loss = model.train_on_batch(x_train, np.asarray(y_train))

            loss += batch_loss

            num_episode_steps += 1

            current_game_state = deepcopy(next_game_state)

        print(i)
        print(loss / num_episode_steps)

        tot_loss[i] = (loss / num_episode_steps)

    print(tot_loss)

    # plot_training_history(tot_loss)

    model.save('./nn_model.h5')

**Tasks:** 

1. Train the model, download the file nn_model.h5. Store it locally in the jupyter folder, and execute play_deep_q_model() in jupyter_main. 


2. Try to improve the model by testing different parameters, such as the neural network architecture, number of hidden neurons, number of hidden layers etc...

In [ ]:
train(level=level, num_training_episodes=100, batch_size=100)